In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np

"Machine learning tools"
import pickle

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier

from classification.datasets import Dataset
from classification.utils.audio_student import AudioUtil, Feature_vector_DS

from classification.utils.plots import (
    plot_decision_boundaries,
    plot_specgram,
    show_confusion_matrix,
)
from classification.utils.utils import accuracy

ModuleNotFoundError: No module named 'classification.datasets'

In [ ]:
np.random.seed(0)

In [ ]:
### TO RUN
dataset = Dataset()
classnames = dataset.list_classes()

print("\n".join(classnames))

In [ ]:
### TO RUN
fm_dir = "data/feature_matrices/"  # where to save the features matrices
new_dataset_dir = "src/classification/datasets/new_dataset/melvecs/"
model_dir = "data/models/"  # where to save the models
os.makedirs(fm_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [ ]:
### TO RUN

"Creation of the dataset"
myds = Feature_vector_DS(dataset, Nft=512, nmel=20, duration=950, shift_pct=0.0)

"Some attributes..."
myds.nmel
myds.duration
myds.shift_pct
myds.sr
myds.data_aug
myds.ncol


idx = 0


In [ ]:
# TRANSFORMATION ON FEATURE VECTOR

def add_noise(feature_vector, snr_db=20):
    """Adds white noise to a feature vector based on the given SNR (Signal-to-Noise Ratio)."""
    power_signal = np.mean(feature_vector ** 2)
    power_noise = power_signal / (10 ** (snr_db / 10))
    noise = np.random.normal(0, np.sqrt(power_noise), feature_vector.shape)
    return feature_vector + noise

def shifting(feature_vector, shift_max=20):
    """Shifts mel spectrogram feature vectors along the time axis by a random shift between 0 and shift_max."""
    shift = np.random.randint(0, shift_max)
    return np.roll(feature_vector, shift, axis=0)  # Rolling along the first axis

In [ ]:
### FEATURE EXTRACTION FROM SOUND
"Random split of 70:30 between training and validation"
train_pct = 0.7

featveclen = len(myds["fire", 0, ""])  # number of items in a feature vector
nitems = len(myds)  # number of sounds in the dataset
naudio = dataset.naudio  # number of audio files in each class
nclass = dataset.nclass  # number of classes
nlearn = round(naudio * train_pct)  # number of sounds among naudio for training

data_aug_factor = 1
class_ids_aug = np.repeat(classnames, naudio * data_aug_factor)

X = np.zeros((data_aug_factor * nclass * naudio, featveclen))
for s in range(data_aug_factor):
    for class_idx, classname in enumerate(classnames):
        for idx in range(naudio):
            featvec = myds[classname, idx, ""]
            X[s * nclass * naudio + class_idx * naudio + idx, :] = featvec
np.save(fm_dir + "X_basic.npy", X)
y = class_ids_aug.copy()
np.save(fm_dir + "y_basic.npy", y)

print(f"Shape of the basic feature matrix : {X.shape}")
print(f"Number of labels : {len(y)}")


# FEATURE EXTRACTION FROM FILE

naudio_new = 40

X_new_dataset = []
y_new_dataset = []
X_new_dataset_aug = []
y_new_dataset_aug = []

for classname in classnames:
    for idx in range(1, naudio_new + 1):  # Assuming files are named from 1 to 40
        file_path = os.path.join(new_dataset_dir, f"{classname}_{idx:02d}.npy")
        if os.path.exists(file_path):
            featvec = np.load(file_path)
            featvec_shifted = shifting(featvec)  # Apply shifting transformation
            featvec_noisy = add_noise(featvec)  # Apply noise transformation
            X_new_dataset.append(featvec.flatten())
            y_new_dataset.append(classname)
            
            X_new_dataset_aug.append(featvec.flatten())
            X_new_dataset_aug.append(featvec_shifted.flatten())
            X_new_dataset_aug.append(featvec_noisy.flatten())
            y_new_dataset_aug.append(classname)
            y_new_dataset_aug.append(classname)
            y_new_dataset_aug.append(classname)
        else:
            print(f"Warning: File {file_path} not found.")

X_new_dataset = np.array(X_new_dataset)
y_new_dataset = np.array(y_new_dataset)
X_new_dataset_aug = np.array(X_new_dataset_aug)
y_new_dataset_aug = np.array(y_new_dataset_aug)

np.save(fm_dir + "X_new_dataset.npy", X_new_dataset)
np.save(fm_dir + "y_new_dataset.npy", y_new_dataset)
np.save(fm_dir + "X_new_dataset_aug.npy", X_new_dataset_aug)
np.save(fm_dir + "y_new_dataset_aug.npy", y_new_dataset_aug)

print(f"Shape of the new feature matrix: {X_new_dataset.shape}")
print(f"Number of labels in new dataset: {len(y_new_dataset)}")

print(f"Shape of the new feature matrix with data augmentation: {X_new_dataset_aug.shape}")
print(f"Number of labels in new dataset with data augmentation: {len(y_new_dataset_aug)}")

We can now create a new augmented dataset and observe if the classification results improve. 

In [ ]:
### AUGMENTED DATASET
list_augmentation = ["original", "noise", "echo", "shifting"]
myds.mod_data_aug(list_augmentation)
print("Number of transformations : ", myds.data_aug_factor)
y_basic_aug = np.repeat(classnames, dataset.naudio * myds.data_aug_factor)
X_basic_aug = np.zeros((myds.data_aug_factor * nclass * naudio, featveclen))

for s in range(len(list_augmentation)):
    aug = list_augmentation[s]
    for idx in range(dataset.naudio):
        for class_idx, classname in enumerate(classnames):
            featvec = myds[classname, idx, aug]
            X_basic_aug[s * nclass * naudio + class_idx * naudio + idx, :] = featvec
            y_basic_aug[s * nclass * naudio + class_idx * naudio + idx] = classname

np.save(fm_dir + "X_basic_aug.npy", X_new_dataset)
np.save(fm_dir + "y_basic_aug.npy", y_new_dataset)

print(f"Shape of the feature matrix : {X_basic_aug.shape}")
print(f"------------------------------------------------------------")
print(f"200 of each transformation. Order : chainsaw1, fire1, fireworks1, gun1, chainsaw2, fire2, ...")


FINAL MODEL SAVE

In [ ]:
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# Directories
fm_dir = "data/feature_matrices/"
model_dir = "data/models/"
os.makedirs(fm_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

# Load datasets

X_new_dataset = np.load(fm_dir + "X_new_dataset.npy")  # Original dataset
y_new_dataset = np.load(fm_dir + "y_new_dataset.npy")

X_new_dataset_aug = np.load(fm_dir + "X_new_dataset_aug.npy")  # Original dataset
y_new_dataset_aug = np.load(fm_dir + "y_new_dataset_aug.npy")


# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_new_dataset, y_new_dataset, test_size=0.3, stratify=y_new_dataset)
X_train_aug, X_test_aug, y_train_aug, y_test_aug = train_test_split(
    X_new_dataset_aug, y_new_dataset_aug, test_size=0.3, stratify=y_new_dataset_aug
)

# ===== TRAIN MODEL WITH PCA =====
pca = PCA(n_components=0.98)  # Keep 98% variance
X_train_aug_pca = pca.fit_transform(X_train_aug)
X_test_aug_pca = pca.transform(X_test_aug)  # Apply the same transformation to test data
print(f"Number of principal features kept: {pca.n_components_}")

# Save the PCA model separately
pca_filename = os.path.join(model_dir, "pca.pickle")
with open(pca_filename, "wb") as pca_file:
    pickle.dump(pca, pca_file)
print(f"\nPCA model saved as {pca_filename}")

# Train the Random Forest model on PCA-reduced features
rf_model_pca = RandomForestClassifier(
    n_estimators=400,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)
rf_model_pca.fit(X_train_aug_pca, y_train_aug)

# Save the model trained **with PCA**
rf_filename_pca = os.path.join(model_dir, "model_pca.pickle")
with open(rf_filename_pca, "wb") as rf_file:
    pickle.dump(rf_model_pca, rf_file)
print(f"\nRandom Forest model WITH PCA saved as {rf_filename_pca}")

# ===== TRAIN MODEL WITHOUT PCA =====
rf_model_no_pca = RandomForestClassifier(
    n_estimators=400,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)
rf_model_no_pca.fit(X_train_aug, y_train_aug)  # Train directly on raw `X_aug` features

# Save the model trained **without PCA**
rf_filename_no_pca = os.path.join(model_dir, "model_without_pca.pickle")
with open(rf_filename_no_pca, "wb") as rf_file:
    pickle.dump(rf_model_no_pca, rf_file)
print(f"\nRandom Forest model WITHOUT PCA saved as {rf_filename_no_pca}")

# ===== TRAIN MODEL WITHOUT DATA TRANSFORMATION (Original Data `X_basic`) =====
rf_model_no_transform = RandomForestClassifier(
    n_estimators=400,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)
rf_model_no_transform.fit(X_train, y_train)  # Train on untransformed data

# Save the model trained **on X_basic (original features)**
rf_filename_no_transform = os.path.join(model_dir, "model_without_data_transformation.pickle")
with open(rf_filename_no_transform, "wb") as rf_file:
    pickle.dump(rf_model_no_transform, rf_file)
print(f"\nRandom Forest model WITHOUT DATA TRANSFORMATION saved as {rf_filename_no_transform}")




# ===== TRAIN MODEL WITH NORMALIZATION =====
X_train_normalized = []
X_test_normalized = []
for i in range(len(X_train)):
    X_train_normalized.append(X_train[i]/np.linalg.norm(X_train[i]))

for i in range(len(X_test)):
    X_test_normalized.append(X_test[i]/np.linalg.norm(X_test[i]))
    

# Train the model on normalized features
rf_model_normalized = RandomForestClassifier(
    n_estimators=400,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)
rf_model_normalized.fit(X_train_normalized, y_train)

# Save the model trained **with normalization**
rf_filename_normalized = os.path.join(model_dir, "model_normalized_no_transformation.pickle")
with open(rf_filename_normalized, "wb") as rf_file:
    pickle.dump(rf_model_normalized, rf_file)
print(f"\nRandom Forest model WITH NORMALIZATION saved as {rf_filename_normalized}")






# ===== EVALUATION FUNCTION (optional but useful) =====
def evaluate_model(model, X_test, y_test, description):
    predict = model.predict(X_test)

    # Compute metrics for each class
    classes = np.unique(y_test)
    precision_per_class = precision_score(y_test, predict, average=None, labels=classes)
    recall_per_class = recall_score(y_test, predict, average=None, labels=classes)

    test_accuracy_per_class = []
    conf_matrix = confusion_matrix(y_test, predict, labels=classes)

    for i, cls in enumerate(classes):
        class_accuracy = conf_matrix[i, i] / conf_matrix[i, :].sum()
        test_accuracy_per_class.append(class_accuracy)

    # Cross-validation accuracy
    cv_scores = cross_val_score(model, X_test, y_test, cv=10, scoring='accuracy')
    mean_cv_accuracy = np.mean(cv_scores)

    # Print results
    print(f"\n=== {description} ===")
    print(f"Test Accuracy (Overall): {np.mean(predict == y_test):.4f}")
    print(f"Mean CV Accuracy: {mean_cv_accuracy:.4f}")

    print("\nPer-Class Metrics:")
    for i, cls in enumerate(classes):
        print(f"Class {cls}:")
        print(f"  Precision: {precision_per_class[i]:.4f}")
        print(f"  Recall: {recall_per_class[i]:.4f}")
        print(f"  Accuracy: {test_accuracy_per_class[i]:.4f}")

# Evaluate all models
evaluate_model(rf_model_pca, X_test_aug_pca, y_test_aug, "Model WITH PCA")
evaluate_model(rf_model_no_pca, X_test, y_test, "Model WITHOUT PCA")
evaluate_model(rf_model_no_transform, X_test, y_test, "Model WITHOUT DATA TRANSFORMATION")
evaluate_model(rf_model_normalized, X_test_normalized, y_test, "Model WITH NORMALIZATION")


MEAN ACCURACY ON 100 ITERATIONS

In [ ]:
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from classification.utils.utils import accuracy

# Directories for saving models
model_dir = "data/models/"
os.makedirs(model_dir, exist_ok=True)

# Ensure dataset (X_aug, y_aug) exists
try:
    X = X_aug
    y = y_aug
except NameError:
    raise ValueError("X_aug and y_aug must be defined before running this script.")

# Number of iterations
num_iterations = 20

# Lists to store scores
accuracy_scores = []
cv_accuracy_scores = []

for i in range(num_iterations):
    print(f"\nIteration {i + 1}/{num_iterations}")

    # Split the dataset into training and testing subsets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=i  # Different splits per iteration
    )

    # Train the Random Forest model
    model = RandomForestClassifier(
        n_estimators=400,
        max_depth=20,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=i  # Different initialization per iteration
    )
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Compute overall accuracy
    test_accuracy = accuracy(y_pred, y_test)
    accuracy_scores.append(test_accuracy)

    # Perform cross-validation on the training set
    cv_scores = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy')
    mean_cv_accuracy = np.mean(cv_scores)
    cv_accuracy_scores.append(mean_cv_accuracy)

    print(f"Test Accuracy: {test_accuracy:.4f} | Mean CV Accuracy: {mean_cv_accuracy:.4f}")

# Compute overall statistics
mean_test_accuracy = np.mean(accuracy_scores)
std_test_accuracy = np.std(accuracy_scores)

mean_cv_accuracy = np.mean(cv_accuracy_scores)
std_cv_accuracy = np.std(cv_accuracy_scores)

# Print final results
print("\n=== FINAL RESULTS AFTER 100 ITERATIONS ===")
print(f"Mean Test Accuracy: {mean_test_accuracy:.4f} ± {std_test_accuracy:.4f}")
print(f"Mean Cross-Validation Accuracy: {mean_cv_accuracy:.4f} ± {std_cv_accuracy:.4f}")
